In [128]:
url = "https://raw.githubusercontent.com/chiarorosa/ia_aprendizado_maquina_basico/main/ml-dataset/kaggle-basico/attrition_Rate_Company.csv"

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score # Avaliação de Acurácia
from sklearn.preprocessing import StandardScaler,LabelEncoder, OneHotEncoder
from sklearn import tree
from sklearn.model_selection import RandomizedSearchCV
import numpy as np


In [129]:
df = pd.read_csv(url)
df.head()

df.dropna(inplace=True)

labelEnconder = LabelEncoder()
categorizacaoColunas = ['BusinessTravel', 'EducationField', 'Department', 'Gender', 'JobRole', 'MaritalStatus','Attrition','Over18']

for coluna in categorizacaoColunas:
    df[coluna] = labelEnconder.fit_transform(df[coluna])
    
X = df.drop('Attrition', axis=1)
y = df['Attrition']







In [130]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(X)


# Dividir em dois conjuntos, treinamento e teste, usando pareto 80/20
X_train, X_test, y_train, y_test = train_test_split(
    x_scaled,
    y,
    test_size=0.2, # pareto
    random_state=42 # reproduzivel
)

In [131]:
grade = {
    'criterion': ['gini', 'entropy'],  # Measure of split quality
    'splitter': ['best', 'random'],    # Strategy for choosing splits
    'max_depth':[None, 10, 20, 30, 40, 50],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],    # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],       # Minimum number of samples required to be at a leaf node
    'max_features': ['sqrt', 'log2', None],  # Number of features to consider for best split
    'random_state': [42]  # For reproducibility
}

Tree = RandomizedSearchCV(
    tree.DecisionTreeClassifier(),
    grade,
    cv=5,
    scoring='accuracy',
    n_iter=20,  # Número de iterações de busca aleatória
    n_jobs=-1,  # Use todos os núcleos disponíveis
)

Tree.fit(X_train, y_train)

melhor = Tree.best_estimator_

y_pred = melhor.predict(X_test)

acuracia = accuracy_score(y_test, y_pred)

print(f'Acurácia: {acuracia * 100:.2f}')
print(f'Melhor modelo: {melhor}')
print(f'Parâmetros do melhor modelo: {Tree.best_params_}')

guardaAcc = 0

with open('Melhor Acurracia Tree.txt', 'r+', encoding='utf-8') as f:
        try:
            linha = f.readline().strip()
            if linha.startswith('Acurácia:'):
                guardaAcc = float(linha.split(': ')[1]) / 100
                print(f'Acurácia anterior: {guardaAcc * 100:.2f}')
                if acuracia > guardaAcc:
                    f.seek(0) # vai pro inicio
                    f.write(f'Acurácia: {acuracia * 100:.2f} \n')
                    
                    f.truncate() #  remove residuos antigos
                    print(f'Acurácia atualizada: {acuracia * 100:.2f}')
                else:
                    print('Acurácia não atualizada')
        except Exception as e:
            print(f"Erro ao ler o arquivo: {e}")
       


Acurácia: 97.21
Melhor modelo: DecisionTreeClassifier(criterion='entropy', max_depth=50, max_features='sqrt',
                       random_state=42)
Parâmetros do melhor modelo: {'splitter': 'best', 'random_state': 42, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 50, 'criterion': 'entropy'}
Acurácia anterior: 99.30
Acurácia não atualizada
